# Refinitiv Data Platform Library for Python
## Content - Pricing - Streaming

This notebook demonstrates how a Price object to receive streaming real-time data.  

## Import the library and load credentials

Credentials used by this and the other tutorials notebooks are stored in the **Configuration/credentials.ipynb** file.     

You should have edited the **Configuration/credentials.ipynb** to set your credentials as part of the **Quick Start** step.

In [1]:
from refinitiv.data.content import pricing
from pandas import DataFrame
from IPython.display import display, clear_output

%run ../../Configuration/credentials.ipynb

## Open the session of your choice

Use our helper function in the Credentials notebook, **open_session(session_type)**, to create and open a session to connect to the 
- Refinitiv Data Platform directly (session_type="rdp") or via 
- Eikon 4 or Refinitiv Workspace (session_type="desktop") or via a 
- local realtime infrastructure (session_type="deployed").

You can also set a default in the **credentials** notebook - which I have done, and so I don't need to specify a session_type:

In [2]:
# I have defined a default Session type in the Credentials notebook - so no need to provide one here
open_session()

### Define a callback to receive streaming data

In [3]:
# declare a DataFrame to use for storing our streaming data
df = DataFrame()

# Function to initially populate Dataframe, once initial values received for all items
def create_dataframe(streaming_prices):
    global df
    snapshot = streaming_prices.get_snapshot()
    field_names = snapshot.columns[1:]
    instrument_names = snapshot['Instrument'].values
    df = DataFrame(index=instrument_names, columns=field_names)
    for price in streaming_prices:
        for field_name, field_value in price.get_fields().items():
            df.at[price.name, field_name] = field_value
    display(df)

# Function to update dataframe, when we receive updates for individual items
def update_dataframe(streaming_prices, instrument_name, fields):
    global df
    clear_output(wait=True)
    for field_name, field_value in fields.items():
        df.at[instrument_name, field_name] = field_value
    display(df)

## Create a stream and open it

In [4]:
# Define our Streaming Price object
streams = rd.content.pricing.Definition(
    ['EUR=', 'GBP=', 'JPY=', 'CAD='],
    fields=['BID', 'ASK']
).get_stream()

# Specify callback handler for any updates
streams.on_update(update_dataframe)
# Callback for when we have received initial values for all items
streams.on_complete(create_dataframe)
# Send the requests to the server and open the streams for all items
streams.open()

,BID,ASK
EUR=,1.2095,1.2099
GBP=,None,None
JPY=,None,None
CAD=,None,None


,BID,ASK
EUR=,1.2095,1.2099
GBP=,1.4103,1.4107
JPY=,109.69,109.7
CAD=,None,None


,BID,ASK
EUR=,1.2095,1.2099
GBP=,1.4103,1.4107
JPY=,109.69,109.7
CAD=,None,None


,BID,ASK
EUR=,1.2095,1.2099
GBP=,1.4103,1.4107
JPY=,109.69,109.7
CAD=,1.2161,1.2165


<StreamState.Open: 3>

## Close the stream

In [5]:
streams.close()

<StreamState.Closed: 1>

## Close the session

In [6]:
close_session()